In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("comments_cutted_cs.csv", encoding='utf-8-sig')

In [3]:
import jieba

In [4]:
def chinese_word_cut(mytext): #分詞
    return " ".join(jieba.cut(mytext))

In [5]:
df["content_cutted"] = df.評論.apply(chinese_word_cut) #分詞

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\a2314\AppData\Local\Temp\jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built succesfully.


In [6]:
df.content_cutted #分詞結果確認

0      標題 [ 遊記 ] 花蓮 旅遊 - 慕谷 慕魚 ( 超 美的 溪谷 ， 享受 天然 的 魚去...
1      標題 [ 遊記 ] 花蓮 統茂 渡假 莊園 - 安靜 休息 的 好 地方 時間 WedOct...
2      標題 [ 遊記 ] 入宿捕 獲 野生 Duncan 時間 SatDec600 : 29 : ...
3      標題 [ 食記 ] 反推 黃 昏市場 汗 馬 X 斯時間 TueAug2623 : 52 :...
4      標題 [ 遊記 ] 民宿 經驗靜 巷裡 的 27 號時間 TueAug2817 : 34 :...
5      標題 [ 食記 ] 一品 坊 早餐 ， 正當冰 很多 喵 星人 時間 ThuNov1015 ...
6      標題 [ 遊記 ] 9 / 5 六十 石 山花 況時間 TueSep623 : 34 : 5...
7      標題 [ 遊記 ] 太魯閣 燕子 口步 道 時間 TueMay2013 : 38 : 282...
8      標題 Fw : [ 食記 ] 花蓮 ： 品味 金針 花 ☆ 六十 石山 鐵掌櫃 茶舍 時間 ...
9      標題 [ 食記 ] 花蓮 市區 — 花蓮 市區 — 痣 男 ｜ 賣 滷 肉 飯 、 賣 麵 ...
10     標題 [ 食記 ] 月 之盧 食堂 梅子 雞 + 來 去 秘密 喝咖啡 時間 MonNov2...
11     標題 [ 食記 ] 花蓮 市區 — 曾品 麻辣 臭豆腐 ｜ 又 臭 又 香 ！ 一口 咬下 ...
12     標題 [ 食記 ] 花蓮 市區 — 安娜 咖啡 ｜ 高 CP 值 下午茶 不到 百元 的 美...
13     標題 [ 食記 ] 花蓮 市區 — 胡仕托 ｜ 鬆 軟 綿 密 的 清爽 系 厚 鬆 餅 時...
14     標題 [ 食記 ] 花蓮翰品 酒店 試婚 菜 時間 MonJul207 : 32 : 152...
15     標題 [ 食記 ] 花蓮 市區 — 半寓 咖啡 ｜ 一間 老屋 乾洗店 一陣 咖啡 飄香 時...
16     標題 [ 遊記 ] 悠悠 哉 哉 的 193 縣道 ( 用 畫畫 分享 景點 ) 時間 We...
17     標題 [ 遊記 ] 蝴蝶谷 溫泉 渡假村 - 東

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#向量化

In [16]:
n_features = 1000 #提取1000個關鍵詞

In [17]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

max_df, min_df: 在建立单词表的时候会取df在[min_df, max_df]区间内的单词。
使用整数时，表示单词出现的document个数，使用浮点数时，表示出现的document个数再除以总的document个数（即频率）。

In [18]:
from sklearn.decomposition import LatentDirichletAllocation #LDA套件

In [24]:
n_topics = 10 #人為設定幾個主題，此範例為10個主題
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [25]:
lda.fit(tf)

C:\Users\a2314\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [22]:
n_top_words = 10 #定每個主題顯示前5個關鍵詞

In [26]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
部落 原住民 大家 阿美族 豐濱 孩子 工作 活動 一起 文化
Topic #1:
咖哩 大地 鳳林 套餐 生態 蜂蜜 自然 教育 大家 小菜
Topic #2:
太魯閣 步道 包心 公園 粉圓 焦糖 公路 國家 五霸 秀林
Topic #3:
玉里 臭豆腐 韭菜 高麗菜 生意 味道 店家 花蓮縣 15 還是
Topic #4:
早餐 蛋餅 06 起司 機會 25 吉安 花蓮市 09 早味
Topic #5:
不過 一個 地方 還是 時候 非常 知道 大家 其實 很多
Topic #6:
夜市 自強 排隊 包子 東大門 美食 口味 烤肉 小吃 公正
Topic #7:
民宿 這裡 金針 房間 六十 一個 住宿 石山 飯店 遊記
Topic #8:
新城 七星 檸檬汁 花蓮縣 天主堂 佳興 冰果 柴魚 博物 松園
Topic #9:
00 好吃 餐廳 店家 非常 味道 起來 口感 營業 咖啡



In [27]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

C:\Users\a2314\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\Users\a2314\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.042222  0.045039       1        1  41.045836
5     -0.124317  0.071862       2        1  22.000999
7     -0.119799  0.037353       3        1  19.277160
0     -0.199421 -0.028731       4        1   6.755293
2     -0.125769  0.102542       5        1   5.059633
6      0.208427  0.181888       6        1   3.250124
8      0.175069  0.126965       7        1   1.291135
1     -0.021258 -0.249299       8        1   0.501354
4      0.180482 -0.207482       9        1   0.497212
3      0.068807 -0.080135      10        1   0.321255, topic_info=     Category        Freq Term       Total  loglift  logprob
term                                                        
538   Default  496.000000   早餐  496.000000  30.0000  30.0000
923   Default  644.000000   部落  644.000000  29.0000  29.0000
442   Default  518.000000   店家  518.000000  28.0000  28.0000
302   Default  451.000000   味道  451.000000  27.0000  27.0000
342   Default  403.000000   夜市  403.000000  26.0000  26.0000
0     Default  972.000000   00  972.000000  25.0000  25.0000
804   Default  535.000000  花蓮縣  535.000000  24.0000  24.0000
803   Default  511.000000  花蓮市  511.000000  23.0000  23.0000
911   Default  544.000000   還是  544.000000  22.0000  22.0000
363   Default  330.000000  太魯閣  330.000000  21.0000  21.0000
1     Default  416.000000   03  416.000000  20.0000  20.0000
347   Default  592.000000   大家  592.000000  19.0000  19.0000
271   Default  450.000000   口味  450.000000  18.0000  18.0000
606   Default  775.000000   民宿  775.000000  17.0000  17.0000
365   Default  171.000000   套餐  171.000000  16.0000  16.0000
771   Default  368.000000   美食  368.000000  15.0000  15.0000
96    Default  188.000000   七星  188.000000  14.0000  14.0000
520   Default  294.000000   料理  294.000000  13.0000  13.0000
303   Default  141.000000   咖哩  141.000000  12.0000  12.0000
260   Default  300.000000  原住民  300.000000  11.0000  11.0000
131   Default  696.000000   不過  696.000000  10.0000  10.0000
521   Default  170.000000   新城  170.000000   9.0000   9.0000
816   Default  138.000000   蛋餅  138.000000   8.0000   8.0000
887   Default  330.000000   這邊  330.000000   7.0000   7.0000
95    Default  318.000000   一點  318.000000   6.0000   6.0000
298   Default  324.000000   名稱  324.000000   5.0000   5.0000
789   Default  201.000000   自強  201.000000   4.0000   4.0000
370   Default  846.000000   好吃  846.000000   3.0000   3.0000
294   Default  247.000000   吉安  247.000000   2.0000   2.0000
304   Default  494.000000   咖啡  494.000000   1.0000   1.0000
...       ...         ...  ...         ...      ...      ...
991   Topic10   13.701674  高麗菜   49.054095   4.4653  -3.3716
688   Topic10    9.834766   生意   73.398166   3.7307  -3.7032
465   Topic10    4.051138  想像中   43.289841   3.3718  -4.5901
996   Topic10    4.489039   鳳林   49.323646   3.3439  -4.4875
827   Topic10    3.609414   要來   52.137095   3.0704  -4.7056
318   Topic10    3.129968   回憶   62.031443   2.7541  -4.8481
604   Topic10    2.361552   比起   47.219275   2.7452  -5.1298
773   Topic10    2.683737   習慣   54.377424   2.7320  -5.0019
137   Topic10    2.497892  中正路   51.028024   2.7238  -5.0737
11    Topic10    5.474603   15  137.562664   2.5167  -4.2890
483   Topic10    1.863656   或許   52.930122   2.3943  -5.3666
15    Topic10    4.278435   18  121.870602   2.3913  -4.5355
157   Topic10    2.198903   人覺   79.240151   2.1562  -5.2012
319   Topic10    1.460707   回程   54.698215   2.1178  -5.6102
864   Topic10    3.733917   路上  161.580212   1.9731  -4.6717
196   Topic10    3.347377   傳統  155.007677   1.9054  -4.7809
508   Topic10    1.493710   攤位   74.235906   1.8347  -5.5878
99    Topic10    1.046637   上去   52.252037   1.8302  -5.9435
434   Topic10    1.037683   年前   53.713660   1.7940  -5.9521
302   Topic10    7.581087   味道  451.535819   1.6537  -3.9635
155   Topic10    0.916271   人家   58.916466   1.5771  -6.0766
193   Topic10  

图的左侧，用圆圈代表不同的主题，圆圈的大小代表了每个主题分别包含文章的数量。

图的右侧，列出了最重要（频率最高）的30个关键词列表。注意当你没有把鼠标悬停在任何主题之上的时候，这30个关键词代表全部文本中提取到的30个最重要关键词。
如果你把鼠标悬停在1号上面：右侧的关键词列表会立即发生变化，红色展示了每个关键词在当前主题下的频率。